# DESCARGA Y CÁLCULO DEL SENTIMIENTO DE NOTICIAS DEL DIARIO EL TIEMPO

Este notebook describe el paso a paso que permitirá calcular el sentimiento de las noticias publicadas en el diario [El Tiempo](https://www.eltiempo.com)

### Paso 1: Instalación de software necesario

--- Python

Puede obtenerse instalando [Anaconda](https://www.anaconda.com). Para ello, ingrese [aquí](https://www.anaconda.com/products/distribution), descargue el instalador y ejecútelo.

--- Git

Ingrese [aquí](https://git-scm.com/downloads), descargue el instalador y ejecútelo.

### Paso 2: Instalación de dependencias

Para ello, ejecute la siguiente celda

In [ ]:
!pip install scrapy==2.6.1
!pip install pyodbc==4.0.32
!pip install nltk==3.6.5
!pip install textblob==0.17.1
!pip install transformers==4.17.0
!pip install pysftp==0.2.9
!pip install ipywidgets==7.6.5
!pip install deep-translator==1.8.3
!pip install sentiment-analysis-spanish==0.0.25
!pip install pysentimiento==0.3.2

### Paso 2.1: Carga de librerías necesarias

Para ello, ejecute la siguiente celda

In [ ]:
import os
import shutil
from IPython.display import clear_output
import pandas as pd
from toolbox import get_date_inputs, clean_news, translate_news, upload_to_lz, save_local

from pysentimiento import create_analyzer
from sentiment_analysis_spanish import sentiment_analysis
import nltk

from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

nltk.download([
  "names",
  "stopwords",
  "state_union",
  "twitter_samples",
  "movie_reviews",
  "averaged_perceptron_tagger",
  "vader_lexicon",
  "punkt",
])

clear_output(wait = True)

analyzer = create_analyzer(task="sentiment", lang="es")
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
sia = SentimentIntensityAnalyzer()
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels = 3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis", model = finbert, tokenizer = tokenizer)

clear_output(wait = False)
print('LIBRERÍAS IMPORTADAS CORRECTAMENTE')

### Paso 3: Descarga de la última versión del código fuente (código que permitirá la descarga de las noticias)

Para ello, es necesario ejecutar la siguiente celda

In [ ]:
!if exist eltiempo_news_scraper (rmdir /s /q eltiempo_news_scraper)

REPOSITORY_URL = 'http://sbmdeqpc04/data-extraction-dasolano/eltiempo_news_scraper'

!git clone $REPOSITORY_URL
clear_output(wait = True)

!if exist eltiempo_news_scraper (echo 'LA ULTIMA VERSION DEL CODIGO FUE DESCARGADA EXITOSAMENTE') else (echo '¡ERROR!: EL CODIGO NO PUDO SER DESCARGADO. REVISE CONEXION Y VPN')

### Paso 4: Descarga de las noticias de interés

Las noticias se pueden filtrar por:


* *Términos de busqueda*  

Inmediatamente después de la siguiente celda usted tendrá la oportunidad de ingresar los términos sobre los cuales quiere buscar noticias. Cada vez que le aparezca
un campo para ingresar texto escriba el término de interés y presione la tecla *enter*. Por favor no escriba palabras que no agregan significado; por ejemplo, en vez de escribir "Banco de la República" escriba "Banco República", o en vez de "Grupo de Energía de Bogotá" escriba "Grupo Energía Bogotá". Si ya escribió todos los términos de interés, en vez de escribir uno adicional presione la tecla *enter*.
En caso de que quiera buscar todas las noticias (es decir, todas y absolutamente todas las que el programa encuentre), simplemente presione la tecla *enter* cuando le aparezca por primera vez el campo de texto.

In [ ]:
SEARCH_TERMS = []

while True:
  search_term = input('Ingrese el término de búsqueda o presione la tecla ENTER para terminar\n').strip().upper()
  if not search_term:
    break
  SEARCH_TERMS.append(search_term)

print('Términos de búsqueda:', SEARCH_TERMS if SEARCH_TERMS else 'TODAS LAS NOTICIAS')

* *Fechas inicial y final*

De la misma manera, inmediatamente después de siguiente celda debe indicar el periodo en el que se quieren buscar noticias. Para ello, ingrese el mes (por ejemplo: "Febrero") y el año (por ejemplo: 2021) inicial y el mes y el año final.

In [ ]:
start_month, start_year, end_month, end_year = get_date_inputs()
START_DATE = f'{start_year}-{start_month}'
END_DATE = f'{end_year}-{end_month}'

In [ ]:
SEARCH_TERMS = ['GRUPO ARGOS', 'BANCO BOGOTA', 'BANCOLOMBIA', 'CEMENTOS ARGOS', 'CELSIA', 'CORFICOLOMBIANA', 'DAVIVIENDA', 'ECOPETROL', 'GRUPO AVAL', 'GRUPO ENERGIA BOGOTA', 'ISA', 'NUTRESA', 'GRUPO SURA']
start_month, start_year, end_month, end_year = 1, 2010, 5, 2022

In [ ]:
!if exist news (rmdir /s /q news)
SEARCH_TERMS_STR = ','.join(SEARCH_TERMS)
current_year = int(start_year)
end_year = int(end_year)

while current_year <= end_year:
  current_start_date = f'{current_year}-{start_month if current_year == int(start_year) else "01"}'
  current_end_date = f'{current_year}-{12 if current_year < end_year else end_month}'
  !cd eltiempo_news_scraper && scrapy crawl news -a search_terms="$SEARCH_TERMS_STR" -a start_date=$current_start_date -a end_date=$current_end_date -o ..\news\news-"$current_year".csv
  clear_output(wait = True)
  current_year += 1

clear_output(wait = False)
!if exist news (echo 'NOTICIAS DESCARGADAS CON EXITO') else (echo '¡ERROR!: ALGO OCURRIO, INTENTE NUEVAMENTE ')

### Paso 5: Limpieza de texto y filtrado de las noticias

Las noticias obtenidas tienen algunos caracteres y textos no deseados. Adicionalmente, algunos resultados están duplicados, así como otros que no coinciden con el término de busqueda empleado en ésta. Para limpiar el texto y filtrar las noticias, ejecute la celda inferior.

In [10]:
for current_year in range(int(start_year), int(end_year) + 1):
  current_news = pd.read_csv(f'news/news-{current_year}.csv')
  current_news = current_news.astype(str)
  
  news_cleaned = clean_news(current_news)
  !del news\news-"$current_year".csv
  news_cleaned.to_csv(f'news/news-{current_year}.csv', index = False)
  print(f'NOTICIAS DEL AÑO {current_year} FUERON LIMPIADAS Y FILTRADAS')

print('*' * 50)
print('LIMPIEZA Y FILTRADO CONCLUYERON EXITOSAMENTE')

NOTICIAS DEL AÑO 2010 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2011 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2012 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2013 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2014 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2015 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2016 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2017 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2018 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2019 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2020 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2021 FUERON LIMPIADAS Y FILTRADAS
NOTICIAS DEL AÑO 2022 FUERON LIMPIADAS Y FILTRADAS
**************************************************
LIMPIEZA Y FILTRADO CONCLUYERON EXITOSAMENTE


### Paso 5.1 (OPCIONAL): Persistencia de las noticias (destino: local)
Las noticias serán guardadas en el mismo directorio de este notebook como **ALL_NEWS.csv**

In [11]:
save_local('news', 'ALL_NEWS')

LAS NOTICIAS FUERON GUARDADAS EXITOSAMENTE COMO ALL_NEWS.csv


### Paso 5.2 (OPCIONAL): Persistencia de las noticias (destino: nube)
En caso de querer subir información a la nube se debe utilizar el método **upload_to_lz()**, en donde *database* es el nombre de la base de datos a la cual usted esté autorizado y *table_name* el nombre con el que la tabla quedará cargada en la nube. Tenga en cuenta que en el mismo directorio de este notebook debe existir un archivo llamado **credentials.json**, debe contener lo siguiente: **{ "user": "su_usuario", "password": "su_contraseña", "server": "sbmdeblze004" }**

In [ ]:
save_local('news', 'ALL_NEWS')
all_news = pd.read_csv('ALL_NEWS.csv')
all_news = all_news.astype(str)

database = 'proceso'
table_name = 'dasolano_eltiempo_news'

upload_to_lz(dataframe = all_news, database = database, table_name = table_name)
clear_output(wait = True)
!del ALL_NEWS.csv
print(f'LAS NOTICIAS HAN SIDO CARGADAS A LA NUBE EXITOSAMENTE. UBICACIÓN LZ: {database}.{table_name}')

### Paso 6: Cálculo del sentimiento en español (título, subtítulo y contenido)

Se calcula el sentimiento del título, subtitulo y contenido de cada noticia usando las librerías [pysentimiento](https://github.com/pysentimiento/pysentimiento)
y [sentiment-spanish](https://github.com/sentiment-analysis-spanish/sentiment-spanish)

In [ ]:
if os.path.exists('news_sentiment_spanish'):
  if os.listdir('news_sentiment_spanish'):
    !rmdir /s /q news_sentiment_spanish
    !mkdir news_sentiment_spanish
else:
  !mkdir news_sentiment_spanish

for current_year in range(int(start_year), int(end_year) + 1):

  news_spanish = pd.read_csv(f'news/news-{current_year}.csv')
  news_spanish = news_spanish.astype(str)

  news_title_sentiment = news_spanish['news_title'].apply(lambda title: analyzer.predict(title).probas)
  news_subtitle_sentiment = news_spanish['news_subtitle'].apply(lambda subtitle: analyzer.predict(subtitle).probas)
  news_content_sentiment = news_spanish['news_text_content'].apply(lambda content: analyzer.predict(content).probas)

  news_spanish['news_title_prob_POS_pysent'] = news_title_sentiment.apply(lambda sentiment: sentiment['POS'])
  news_spanish['news_title_prob_NEU_pysent'] = news_title_sentiment.apply(lambda sentiment: sentiment['NEU'])
  news_spanish['news_title_prob_NEG_pysent'] = news_title_sentiment.apply(lambda sentiment: sentiment['NEG'])

  news_spanish['news_subtitle_prob_POS_pysent'] = news_subtitle_sentiment.apply(lambda sentiment: sentiment['POS'])
  news_spanish['news_subtitle_prob_NEU_pysent'] = news_subtitle_sentiment.apply(lambda sentiment: sentiment['NEU'])
  news_spanish['news_subtitle_prob_NEG_pysent'] = news_subtitle_sentiment.apply(lambda sentiment: sentiment['NEG'])

  news_spanish['news_text_content_prob_POS_pysent'] = news_content_sentiment.apply(lambda sentiment: sentiment['POS'])
  news_spanish['news_text_content_prob_NEU_pysent'] = news_content_sentiment.apply(lambda sentiment: sentiment['NEU'])
  news_spanish['news_text_content_prob_NEG_pysent'] = news_content_sentiment.apply(lambda sentiment: sentiment['NEG'])

  news_spanish['news_title_sent_sentspanish'] = news_spanish['news_title'].apply(lambda title: sentiment.sentiment(title))
  news_spanish['news_subtitle_sent_sentspanish'] = news_spanish['news_subtitle'].apply(lambda subtitle: sentiment.sentiment(subtitle))
  news_spanish['news_text_content_sent_sentspanish'] = news_spanish['news_text_content'].apply(lambda content: sentiment.sentiment(content))

  news_spanish.to_csv(f'news_sentiment_spanish/news_sentiment_spanish-{current_year}.csv', index = False)
  
  print(f'CALCULO DE SENTIMIENTO (ESPAÑOL) DE NOTICIAS DEL AÑO {current_year} REALIZADO EXITOSAMENTE')

print('*' * 80)
print('CALCULO DE SENTIMIENTO (ESPAÑOL) REALIZADO EXITOSAMENTE')

### Paso 6.1 (OPCIONAL): Persistencia de las noticias en español con sentimiento (destino: local)
Las noticias serán guardadas en el mismo directorio de este notebook como **ALL_NEWS_SENTIMENT.csv**

In [ ]:
save_local('news_sentiment_spanish', 'ALL_NEWS_SENTIMENT')

### Paso 6.2 (OPCIONAL): Persistencia de las noticias en español con sentimiento (destino: nube)
En caso de querer subir información a la nube se debe utilizar el método **upload_to_lz()**, en donde *database* es el nombre de la base de datos a la cual usted esté autorizado y *table_name* el nombre con el que la tabla quedará cargada en la nube. Tenga en cuenta que en el mismo directorio de este notebook debe existir un archivo llamado **credentials.json**, debe contener lo siguiente: **{ "user": "su_usuario", "password": "su_contraseña", "server": "sbmdeblze004" }**

In [ ]:
save_local('news_sentiment_spanish', 'ALL_NEWS_SENTIMENT')
all_news = pd.read_csv('ALL_NEWS_SENTIMENT.csv')
all_news = all_news.astype(str)

database = 'proceso'
table_name = 'dasolano_eltiempo_news_sentiment'

upload_to_lz(dataframe = all_news, database = database, table_name = table_name)
clear_output(wait = True)
!del ALL_NEWS_SENTIMENT.csv
print(f'LAS NOTICIAS HAN SIDO CARGADAS A LA NUBE EXITOSAMENTE. UBICACIÓN LZ: {database}.{table_name}')

### Paso 7: Cálculo del sentimiento en español (título, subtítulo y contenido unidos)

Se calcula el sentimiento de la unión (concatenación) del título, subtitulo y contenido de cada noticia usando las librerías [pysentimiento](https://github.com/pysentimiento/pysentimiento)
y [sentiment-spanish](https://github.com/sentiment-analysis-spanish/sentiment-spanish). El resultado se guardará en un archivo (en el mismo directorio de este notebook) llamado **news_sentiment_spanish_joined.csv**. Para ello, ejecute la siguiente celda

In [ ]:
if os.path.exists('news_sentiment_spanish_joined'):
  if os.listdir('news_sentiment_spanish_joined'):
    !rmdir /s /q news_sentiment_spanish_joined
    !mkdir news_sentiment_spanish_joined
else:
  !mkdir news_sentiment_spanish_joined
  
for current_year in range(int(start_year), int(end_year) + 1):

  news_spanish = pd.read_csv(f'news/news-{current_year}.csv')
  news_spanish = news_spanish.astype(str)

  news_spanish['news_joined'] = news_spanish['news_title'] + '. ' + news_spanish['news_subtitle'] + '. ' + news_spanish['news_text_content']

  del news_spanish['news_title']
  del news_spanish['news_subtitle']
  del news_spanish['news_text_content']

  news_spanish_joined = news_spanish['news_joined'].apply(lambda news: analyzer.predict(news).probas)

  news_spanish['news_joined_prob_POS_pysent'] = news_spanish_joined.apply(lambda sentiment: sentiment['POS'])
  news_spanish['news_joined_prob_NEU_pysent'] = news_spanish_joined.apply(lambda sentiment: sentiment['NEU'])
  news_spanish['news_joined_prob_NEG_pysent'] = news_spanish_joined.apply(lambda sentiment: sentiment['NEG'])

  news_spanish['news_joined_sent_sentspanish'] = news_spanish['news_joined'].apply(lambda joined_text: sentiment.sentiment(joined_text))

  news_spanish.to_csv(f'news_sentiment_spanish_joined/news_sentiment_spanish_joined-{current_year}.csv', index = False)

  print(f'CALCULO DE SENTIMIENTO (ESPAÑOL) DE NOTICIAS DEL AÑO {current_year} REALIZADO EXITOSAMENTE')

print('*' * 80)
print('CALCULO DE SENTIMIENTO (ESPAÑOL) REALIZADO EXITOSAMENTE')

### Paso 7.1 (OPCIONAL): Persistencia de las noticias en español con sentimiento titulo + subtitulo + contenido (destino: local)
Las noticias serán guardadas en el mismo directorio de este notebook como **ALL_NEWS_SENTIMENT_JOINED.csv**

In [ ]:
save_local('news_sentiment_spanish_joined', 'ALL_NEWS_SENTIMENT_JOINED')

### Paso 7.2 (OPCIONAL): Persistencia de las noticias en español con sentimiento titulo + subtitulo + contenido (destino: nube)
En caso de querer subir información a la nube se debe utilizar el método **upload_to_lz()**, en donde *database* es el nombre de la base de datos a la cual usted esté autorizado y *table_name* el nombre con el que la tabla quedará cargada en la nube. Tenga en cuenta que en el mismo directorio de este notebook debe existir un archivo llamado **credentials.json**, debe contener lo siguiente: **{ "user": "su_usuario", "password": "su_contraseña", "server": "sbmdeblze004" }**

In [ ]:
save_local('news_sentiment_spanish_joined', 'ALL_NEWS_SENTIMENT_JOINED')
all_news = pd.read_csv('ALL_NEWS_SENTIMENT_JOINED.csv')
all_news = all_news.astype(str)

database = 'proceso'
table_name = 'dasolano_eltiempo_news_sentiment_joined'

upload_to_lz(dataframe = all_news, database = database, table_name = table_name)
clear_output(wait = True)
!del ALL_NEWS_SENTIMENT_JOINED.csv
print(f'LAS NOTICIAS HAN SIDO CARGADAS A LA NUBE EXITOSAMENTE. UBICACIÓN LZ: {database}.{table_name}')

### Paso 8: Traducción a inglés de las noticias

Traducción a inglés de las noticias usando la librería [deep-translator](https://github.com/nidhaloff/deep-translator). El resultado se guardará en un archivo (en el mismo directorio de este notebook) llamado **search_results_cleaned_english.csv**. Para ello, ejecute la siguiente celda

In [12]:
if os.path.exists('news_english'):
  if os.listdir('news_english'):
    !rmdir /s /q news_english
    !mkdir news_english
else:
  !mkdir news_english

for current_year in range(int(start_year), int(end_year) + 1):
  news_spanish = pd.read_csv(f'news/news-{current_year}.csv')
  news_spanish = news_spanish.astype(str)

  news_location = f'news_english/news_english-{current_year}'

  news_english = translate_news(news_spanish, news_location)
  news_english.to_csv(f'{news_location}.csv', index = False)
  shutil.rmtree(news_location)

news_english/news_english-2010-0
news_english/news_english-2010-1
news_english/news_english-2010-2
news_english/news_english-2010-3
news_english/news_english-2010-4
news_english/news_english-2010-5
news_english/news_english-2010-6
news_english/news_english-2010-7
news_english/news_english-2010-8
news_english/news_english-2010-9
news_english/news_english-2010-10
news_english/news_english-2010-11
news_english/news_english-2010-12
news_english/news_english-2010-13
news_english/news_english-2010-14
news_english/news_english-2010-15
news_english/news_english-2010-16
news_english/news_english-2010-17
news_english/news_english-2010-18
news_english/news_english-2010-19
news_english/news_english-2010-20
news_english/news_english-2010-21
news_english/news_english-2010-22
news_english/news_english-2010-23
news_english/news_english-2010-24
news_english/news_english-2010-25
news_english/news_english-2010-26
news_english/news_english-2010-27
news_english/news_english-2010-28
news_english/news_englis

### Paso 8.1 (OPCIONAL): Persistencia de las noticias (destino: local)
Las noticias serán guardadas en el mismo directorio de este notebook como **ALL_NEWS_ENGLISH.csv**

In [ ]:
save_local('news_english', 'ALL_NEWS_ENGLISH')

### Paso 8.2 (OPCIONAL): Persistencia de las noticias (destino: nube)
En caso de querer subir información a la nube se debe utilizar el método **upload_to_lz()**, en donde *database* es el nombre de la base de datos a la cual usted esté autorizado y *table_name* el nombre con el que la tabla quedará cargada en la nube. Tenga en cuenta que en el mismo directorio de este notebook debe existir un archivo llamado **credentials.json**, debe contener lo siguiente: **{ "user": "su_usuario", "password": "su_contraseña", "server": "sbmdeblze004" }**

In [ ]:
save_local('news_english', 'ALL_NEWS_ENGLISH')
all_news = pd.read_csv('ALL_NEWS_ENGLISH.csv')
all_news = all_news.astype(str)

database = 'proceso'
table_name = 'dasolano_eltiempo_news_english'

upload_to_lz(dataframe = all_news, database = database, table_name = table_name)
clear_output(wait = True)
!del ALL_NEWS_ENGLISH.csv
print(f'LAS NOTICIAS HAN SIDO CARGADAS A LA NUBE EXITOSAMENTE. UBICACIÓN LZ: {database}.{table_name}')

### Paso 9: Cálculo del sentimiento en inglés (título, subtítulo y contenido)

Se calcula el sentimiento del título, subtitulo y contenido de cada noticia usando las librerías [NLTK](https://www.nltk.org/), [TextBlob](https://textblob.readthedocs.io/en/dev/) y [FinBert](https://huggingface.co/yiyanghkust/finbert-tone). El resultado se guardará en un archivo (en el mismo directorio de este notebook) llamado **news_sentiment_english.csv**. Para ello, ejecute la siguiente celda

In [ ]:
if os.path.exists('news_sentiment_english'):
  if os.listdir('news_sentiment_english'):
    !rmdir /s /q news_sentiment_english
    !mkdir news_sentiment_english
else:
  !mkdir news_sentiment_english

for current_year in range(int(start_year), int(end_year) + 1):
    news_english = pd.read_csv(f'news_english/news_english-{current_year}.csv')
    news_english = news_english.astype(str)

    #NLTK

    news_english_title_nltk = news_english['news_title_english'].apply(lambda title: sia.polarity_scores(str(title)))
    news_english_subtitle_nltk = news_english['news_subtitle_english'].apply(lambda subtitle: sia.polarity_scores(str(subtitle)))
    news_english_content_nltk = news_english['news_text_content_english'].apply(lambda content: sia.polarity_scores(str(content)))

    news_english['news_title_english_polarity_POS_nltk'] = news_english_title_nltk.apply(lambda sentiment: sentiment['pos'])
    news_english['news_title_english_polarity_NEU_nltk'] = news_english_title_nltk.apply(lambda sentiment: sentiment['neu'])
    news_english['news_title_english_polarity_NEG_nltk'] = news_english_title_nltk.apply(lambda sentiment: sentiment['neg'])
    news_english['news_title_english_polarity_COM_nltk'] = news_english_title_nltk.apply(lambda sentiment: sentiment['compound'])

    news_english['news_subtitle_english_polarity_POS_nltk'] = news_english_subtitle_nltk.apply(lambda sentiment: sentiment['pos'])
    news_english['news_subtitle_english_polarity_NEU_nltk'] = news_english_subtitle_nltk.apply(lambda sentiment: sentiment['neu'])
    news_english['news_subtitle_english_polarity_NEG_nltk'] = news_english_subtitle_nltk.apply(lambda sentiment: sentiment['neg'])
    news_english['news_subtitle_english_polarity_COM_nltk'] = news_english_subtitle_nltk.apply(lambda sentiment: sentiment['compound'])

    news_english['news_text_content_english_polarity_POS_nltk'] = news_english_content_nltk.apply(lambda sentiment: sentiment['pos'])
    news_english['news_text_content_english_polarity_NEU_nltk'] = news_english_content_nltk.apply(lambda sentiment: sentiment['neu'])
    news_english['news_text_content_english_polarity_NEG_nltk'] = news_english_content_nltk.apply(lambda sentiment: sentiment['neg'])
    news_english['news_text_content_english_polarity_COM_nltk'] = news_english_content_nltk.apply(lambda sentiment: sentiment['compound'])

    #TEXTBLOB

    news_english_title_textblob = news_english['news_title_english'].apply(lambda title: TextBlob(str(title)).sentiment)
    news_english_subtitle_textblob = news_english['news_subtitle_english'].apply(lambda subtitle: TextBlob(str(subtitle)).sentiment)
    news_english_content_textblob = news_english['news_text_content_english'].apply(lambda content: TextBlob(str(content)).sentiment)

    news_english['news_title_english_pol_textblob'] = news_english_title_textblob.apply(lambda sentiment: sentiment.polarity)
    news_english['news_title_english_sub_textblob'] = news_english_title_textblob.apply(lambda sentiment: sentiment.subjectivity)

    news_english['news_subtitle_english_pol_textblob'] = news_english_subtitle_textblob.apply(lambda sentiment: sentiment.polarity)
    news_english['news_subtitle_english_sub_textblob'] = news_english_subtitle_textblob.apply(lambda sentiment: sentiment.subjectivity)

    news_english['news_text_content_english_pol_textblob'] = news_english_content_textblob.apply(lambda sentiment: sentiment.polarity)
    news_english['news_text_content_english_sub_textblob'] = news_english_content_textblob.apply(lambda sentiment: sentiment.subjectivity)

    #FINBERT

    news_english_title_textblob = news_english['news_title_english'].apply(lambda title: nlp(str(title)))
    news_english_subtitle_textblob = news_english['news_subtitle_english'].apply(lambda subtitle: nlp(str(subtitle)))

    news_english['news_title_english_label_finbert'] = news_english_title_textblob.apply(lambda sentiment: sentiment[0]['label'])
    news_english['news_title_english_label_score_finbert'] = news_english_title_textblob.apply(lambda sentiment: sentiment[0]['score'])

    news_english['news_subtitle_english_label_finbert'] = news_english_subtitle_textblob.apply(lambda sentiment: sentiment[0]['label'])
    news_english['news_subtitle_english_label_score_finbert'] = news_english_subtitle_textblob.apply(lambda sentiment: sentiment[0]['score'])

    news_english.to_csv(f'./news_sentiment_english/news_sentiment_english-{current_year}.csv', index = False)

    print(f'CALCULO DE SENTIMIENTO (INGLÉS) DE NOTICIAS DEL AÑO {current_year} REALIZADO EXITOSAMENTE')

print('*' * 80)
print('CALCULO DE SENTIMIENTO (INGLÉS) REALIZADO EXITOSAMENTE')

### Paso 9.1 (OPCIONAL): Persistencia de las noticias en inglés con sentimiento (destino: local)
Las noticias serán guardadas en el mismo directorio de este notebook como **ALL_NEWS_SENTIMENT_ENGLISH.csv**

In [ ]:
save_local('news_sentiment_english', 'ALL_NEWS_SENTIMENT_ENGLISH')

### Paso 9.2 (OPCIONAL): Persistencia de las noticias en inglés con sentimiento (destino: nube)
En caso de querer subir información a la nube se debe utilizar el método **upload_to_lz()**, en donde *database* es el nombre de la base de datos a la cual usted esté autorizado y *table_name* el nombre con el que la tabla quedará cargada en la nube. Tenga en cuenta que en el mismo directorio de este notebook debe existir un archivo llamado **credentials.json**, debe contener lo siguiente: **{ "user": "su_usuario", "password": "su_contraseña", "server": "sbmdeblze004" }**

In [ ]:
save_local('news_sentiment_english', 'ALL_NEWS_SENTIMENT_ENGLISH')
all_news = pd.read_csv('ALL_NEWS_SENTIMENT_ENGLISH.csv')
all_news = all_news.astype(str)

database = 'proceso'
table_name = 'dasolano_eltiempo_news_sentiment_english'

upload_to_lz(dataframe = all_news, database = database, table_name = table_name)
clear_output(wait = True)
!del ALL_NEWS_SENTIMENT_ENGLISH.csv
print(f'LAS NOTICIAS HAN SIDO CARGADAS A LA NUBE EXITOSAMENTE. UBICACIÓN LZ: {database}.{table_name}')